In [1]:
# Importation des modules

import pandas as pd
from gensim.models import Word2Vec
import random
import matplotlib.pyplot as plt
import numpy as np
import plotly.express as px
import datetime

#Affichage de toutes les colonnes

pd.set_option('display.max_columns', 500)

In [8]:
w2v_model_all = Word2Vec.load('results/word2vec_all_300.model')

In [3]:
df0 = pd.read_csv("data/data_cleaned_NLP.csv", sep = ',', encoding = 'latin-1')

In [4]:
df = df0[['Date', 'Groupe', 'tokenized_replique']].copy()

In [5]:
df['days'] = df.apply(lambda row : (datetime.date(int(row.Date[:4]),int(row.Date[5:7]),int(row.Date[8:])) - 
                    datetime.date(2017,6,28)).days, axis = 1)

In [6]:
df

,Date,Groupe,tokenized_replique,days
0,2019-11-04,Exp,si donnez avis favorable madame secrétaire éta...,859
1,2019-11-04,Exp,plus généralement soutiendrai autre amendement...,859
2,2019-07-02,Exp,question adresse françois rugy ministre état m...,734
3,2018-02-01,Exp,NaN,218
4,2020-05-08,Exp,cette autorisation strictement encadrée décret...,1045
...,...,...,...,...
499838,2018-11-05,Novice,rapporteure avis commission développement dura...,495
499839,2019-09-12,Novice,rapporteure avis défavorable agents sociétés s...,806
499840,2018-02-20,Novice,selon arcep opérateurs dépenseront milliards e...,237
499841,2019-06-14,Novice,rapporteure comprends préoccupation nécessité ...,716


In [10]:
def add_groupe(L, etiquette):
    return [i + etiquette for i in L]

In [11]:
def calcule_WMD_intervalle(d1, d2):
    
    df1 = df[df['days'] >= d1]
    df1 = df1[df1['days'] < d2]
    
    df_Novice = df1[df1['Groupe'] == 'Novice']
    df_Exp = df1[df1['Groupe'] == 'Exp']
    
    try:
        df_Novice = df1[df1['Groupe'] == 'Novice'].sample(50).dropna()
        df_Exp = df1[df1['Groupe'] == 'Exp'].sample(50).dropna()
        print(df_Exp.shape, df_Novice.shape)
    except:
        return np.nan
    
    phrases_Exp = [df_Exp.iloc[i]['tokenized_replique'].split(' ') for i in range(df_Exp.shape[0])]
    phrases_Novice = [df_Novice.iloc[i]['tokenized_replique'].split(' ') for i in range(df_Novice.shape[0])]
    
    WMD = []
    

    for i in phrases_Novice:
        value = np.mean([w2v_model_all.wv.wmdistance(add_groupe(i, '_Novice'), 
                                                     add_groupe(j, '_Exp')) for j in phrases_Exp])
        WMD.append(value)

    return np.mean(WMD)

In [12]:
calcule_WMD_intervalle(0, 15)

(50, 4) (50, 4)


1.514262294860726

In [22]:
#L = [calcule_WMD_intervalle(i, i + 14) for i in range(0, 1094, 14)]
#pd.DataFrame(L).to_csv('results/WMDinter_temporel0.csv')

L = list(pd.read_csv('results/WMDinter_temporel.csv').iloc[0])[1:]

In [28]:
px.line([i for i in L if not(str(i) in ['nan', 'inf'])])

In [24]:
M = [i for i in L if not(str(i) in ['nan', 'inf'])]

In [25]:
px.line([np.mean(M[:i]) for i in range(1,len(M))])